In [11]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
import os
import pickle
%matplotlib inline
from datetime import datetime
from dateutil.parser import parse

In [12]:
with open("all_data.pkl", 'rb') as picklefile: 
    merge_housing = pickle.load(picklefile)
merge_housing.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,Date,count_news_seattle+shooting,count_news_seattle+murder,count_news_seattle+dead+shooting,Open,High,Low,Close,Adj Close,Volume
0,Studio,2011,03,1097.5,2011,03,366200.0,45,32,25,...,2011-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Studio,2011,04,1095.0,2011,04,370800.0,48,27,22,...,2011-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Studio,2011,05,1025.0,2011,05,376000.0,46,32,24,...,2011-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Studio,2011,06,995.0,2011,06,382800.0,37,33,24,...,2011-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Studio,2011,07,1000.0,2011,07,373500.0,33,22,16,...,2011-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:

merge_housing['target_Apartment_rent'] = merge_housing['Apartment_rent']

merge_housing = merge_housing.dropna()
merge_housing.Year = merge_housing.Year.astype(int)
merge_housing.Month = merge_housing.Month.astype(int)
merge_housing.Search_Count_SeattleRent = merge_housing.Search_Count_SeattleRent.astype(int)
merge_housing.Search_Count_SeattleHousing = merge_housing.Search_Count_SeattleHousing.astype(int)
merge_housing.Search_Count_seattleApartment = merge_housing.Search_Count_seattleApartment.astype(int)
merge_housing['Avg Sale To List Mom'] = merge_housing['Avg Sale To List Mom'].str.replace('%','').astype(float)
merge_housing['Avg Sale To List Yoy'] = merge_housing['Avg Sale To List Yoy'].str.replace('%','').astype(float)
merge_housing['New Listings Mom'] = merge_housing['New Listings Mom'].str.replace('%','').astype(float)
merge_housing['New Listings Yoy'] = merge_housing['New Listings Yoy'].str.replace('%','').astype(float)
merge_housing['Median Sale Price Mom'] = merge_housing['Median Sale Price Mom'].str.replace('%','').astype(float)
merge_housing['Median Sale Price Yoy'] = merge_housing['Median Sale Price Yoy'].str.replace('%','').astype(float)
merge_housing['Median Sale Price'] = merge_housing['Median Sale Price'].str.replace('$','').str.replace('K','000').astype(float)
merge_housing['Median List Price'] = merge_housing['Median List Price'].str.replace(',','').astype(float)
merge_housing['New Listings'] = merge_housing['New Listings'].str.replace(',','').astype(float)


merge_housing['Homes Sold'] = merge_housing['Homes Sold'].str.replace(',','').astype(int)
merge_housing['Homes Sold Mom'] = merge_housing['Homes Sold Mom'].str.replace('%','').astype(float)
merge_housing['Homes Sold Yoy'] = merge_housing['Homes Sold Yoy'].str.replace('%','').astype(float)

merge_housing['Inventory'] = merge_housing['Inventory'].str.replace(',','').astype(int)
merge_housing['Inventory Mom'] = merge_housing['Inventory Mom'].str.replace('%','').astype(float)
merge_housing['Inventory Yoy'] = merge_housing['Inventory Yoy'].str.replace('%','').astype(float)
transfer_column = {k:k.replace(' ','') for k in merge_housing.iloc[:,6:-1].columns}


merge_housing = merge_housing.rename(columns=transfer_column)



In [14]:
merge_housing['count_news_seattle+shooting'] = merge_housing['count_news_seattle+shooting'].astype(int)
merge_housing['count_news_seattle+murder'] = merge_housing['count_news_seattle+murder'].astype(int)
merge_housing['count_news_seattle+dead+shooting'] = merge_housing['count_news_seattle+dead+shooting'].astype(int)

In [15]:
merge_housing.dtypes

BedSize                                     object
Year                                         int64
Month                                        int64
Apartment_rent                             float64
Shift_Year                                  object
Shift_Month                                 object
Zillow_Housing_price                       float64
Search_Count_SeattleRent                     int64
Search_Count_SeattleHousing                  int64
Search_Count_seattleApartment                int64
AvgSaleToList                              float64
AvgSaleToListMom                           float64
AvgSaleToListYoy                           float64
HomesSold                                    int64
HomesSoldMom                               float64
HomesSoldYoy                               float64
Inventory                                    int64
InventoryMom                               float64
InventoryYoy                               float64
MedianDom                      

In [16]:
# Save Data to Locale

In [17]:
with open('all_clean_data.pkl', 'wb') as picklefile:
    pickle.dump(merge_housing, picklefile)

In [18]:
with open("all_clean_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)
my_old_data.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,count_news_seattle+shooting,count_news_seattle+murder,count_news_seattle+dead+shooting,Open,High,Low,Close,AdjClose,Volume,target_Apartment_rent
31,Studio,2013,10,1180.0,2013,10,430600.0,40,25,22,...,8170,10900,5460,398.799988,408.059998,357.760010,358.690002,358.690002,81011900.0,1180.0
32,Studio,2013,11,1205.0,2013,11,435300.0,37,23,24,...,9740,13900,6380,358.980011,365.869995,337.730011,362.100006,362.100006,83009000.0,1205.0
33,Studio,2013,12,1208.0,2013,12,430100.0,42,27,27,...,4420,7230,3030,358.739990,383.109985,330.880005,336.369995,336.369995,75253500.0,1208.0
34,Studio,2014,1,1205.0,2014,01,424600.0,40,27,26,...,10400,14800,6720,338.089996,348.299988,288.000000,304.130005,304.130005,138278700.0,1205.0
35,Studio,2014,2,1200.0,2014,02,424900.0,46,27,28,...,10500,15900,7070,304.130005,314.869995,284.380005,312.549988,312.549988,78579200.0,1200.0


In [19]:
my_old_data.dtypes

BedSize                                     object
Year                                         int64
Month                                        int64
Apartment_rent                             float64
Shift_Year                                  object
Shift_Month                                 object
Zillow_Housing_price                       float64
Search_Count_SeattleRent                     int64
Search_Count_SeattleHousing                  int64
Search_Count_seattleApartment                int64
AvgSaleToList                              float64
AvgSaleToListMom                           float64
AvgSaleToListYoy                           float64
HomesSold                                    int64
HomesSoldMom                               float64
HomesSoldYoy                               float64
Inventory                                    int64
InventoryMom                               float64
InventoryYoy                               float64
MedianDom                      